**Twitter Elections Jake**<br>
*Compsci 209 Final Project*

Follow the instructions here for pulling tweets:<br>
https://developer.twitter.com/en/docs/tweets/search/api-reference/premium-search#DataEndpoint

Tutorial for pulling geography of tweets:<br>
https://stackabuse.com/accessing-the-twitter-api-with-python/

Query Parameter:<br>
https://developer.twitter.com/en/docs/tweets/search/api-reference/get-search-tweets

----

In [2]:
from twython import Twython
import pandas as pd
import sys

In [4]:
import config_twitter

app_key = config_twitter.api_key
app_secret_key = config_twitter.api_secret_key
access_token = config_twitter.access_token
access_token_secret = config_twitter.access_token_secret

user_agent = "Jake_Schneider"

In [5]:
twitter = Twython(app_key, app_secret_key,
                  access_token, access_token_secret)

In [6]:
twitter.get_application_rate_limit_status()['resources']['search']

{'/search/tweets': {'limit': 180, 'remaining': 180, 'reset': 1574130206}}

In [7]:
twitter.verify_credentials()

{'id': 1293308917,
 'id_str': '1293308917',
 'name': 'Jake Schneider',
 'screen_name': '_jwschneider',
 'location': 'Cambridge, MA',
 'description': 'Economist. Data Scientist. Entrepreneur. \nFounder @SchneiderEconomics. Founder @PolicyViews. MPA/ID Candidate @Harvard @Kennedy_School.',
 'url': 'https://t.co/KiRNKCPigk',
 'entities': {'url': {'urls': [{'url': 'https://t.co/KiRNKCPigk',
     'expanded_url': 'http://www.policyviews.com',
     'display_url': 'policyviews.com',
     'indices': [0, 23]}]},
  'description': {'urls': []}},
 'protected': False,
 'followers_count': 56,
 'friends_count': 168,
 'listed_count': 0,
 'created_at': 'Sun Mar 24 02:33:01 +0000 2013',
 'favourites_count': 46,
 'utc_offset': None,
 'time_zone': None,
 'geo_enabled': False,
 'verified': False,
 'statuses_count': 57,
 'lang': None,
 'status': {'created_at': 'Fri Oct 11 16:06:20 +0000 2019',
  'id': 1182688895848214528,
  'id_str': '1182688895848214528',
  'text': "Tune in next Friday for the launch of #Po

----

**Search for Conservative Tweets**

In [88]:
# Create query
query = {'q': 'Conservative',
        #'geocode': '37.0902, -95.7129, 1500mi',
        'result_type': 'popular',
        'count': 1000,
        'lang': 'en',
#        'until': "2018-11-06" 
        }

print(query)

{'q': 'Conservative', 'result_type': 'popular', 'count': 1000, 'lang': 'en', 'until': '2018-11-06'}


In [89]:
twitter.get_application_rate_limit_status()['resources']['search']

{'/search/tweets': {'limit': 180, 'remaining': 178, 'reset': 1574176276}}

In [90]:
# Search tweets
dict_ = {'user': [], 'date': [], 'text': [], 'favorite_count': [], 'user_loc': []}
for status in twitter.search(**query)['statuses']:
    dict_['user'].append(status['user']['screen_name'])
    dict_['date'].append(status['created_at'])
    dict_['text'].append(status['text'])
    dict_['favorite_count'].append(status['favorite_count'])
    dict_['user_loc'].append(status['user']['location'])


    
print(len(dict_["user"]))

0


In [91]:
# Structure data in a pandas DataFrame for easier manipulation
df = pd.DataFrame(dict_)
df.sort_values(by='favorite_count', inplace=True, ascending=False)
df.head(100)

,user,date,text,favorite_count,user_loc


In [83]:
# Geographic Heat Map

from geopy.geocoders import Nominatim
import gmplot

geolocator = Nominatim()

# Go through all tweets and add locations to 'coordinates' dictionary
coordinates = {'latitude': [], 'longitude': []}
for count, user_loc in enumerate(df.user_loc):
    try:
        location = geolocator.geocode(user_loc)
        
        # If coordinates are found for location
        if location:
            coordinates['latitude'].append(location.latitude)
            coordinates['longitude'].append(location.longitude)
            
    # If too many connection requests
    except:
        pass
    
# Instantiate and center a GoogleMapPlotter object to show our map
gmap = gmplot.GoogleMapPlotter(30, 0, 3)

# Insert points on the map passing a list of latitudes and longitudes
gmap.heatmap(coordinates['latitude'], coordinates['longitude'], radius=20)

# Save the map to html file
gmap.draw("python_heatmap.html")

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  


In [84]:
from twython import TwythonStreamer
import csv

# Filter out unwanted data
def process_tweet(tweet):
    d = {}
    d['hashtags'] = [hashtag['text'] for hashtag in tweet['entities']['hashtags']]
    d['text'] = tweet['text']
    d['user'] = tweet['user']['screen_name']
    d['user_loc'] = tweet['user']['location']
    return d
    
    
# Create a class that inherits TwythonStreamer
class MyStreamer(TwythonStreamer):     

    # Received data
    def on_success(self, data):

        # Only collect tweets in English
        if data['lang'] == 'en':
            tweet_data = process_tweet(data)
            print(tweet_data)
#            self.save_to_csv(tweet_data)
            return tweet_data

    # Problem with the API
    def on_error(self, status_code, data):
        print(status_code, data)
        self.disconnect()
        
    # Save each tweet to csv file
#    def save_to_csv(self, tweet):
#        with open(r'saved_tweets_conservative.csv', 'a') as file:
#            writer = csv.writer(file)
#            writer.writerow(list(tweet.values()))

In [85]:
# Instantiate from our streaming class
stream = MyStreamer(app_key, app_secret_key,
                  access_token, access_token_secret)
# Start the stream
tweets = stream.statuses.filter(track='conservative')

{'hashtags': [], 'text': 'RT @BorisJohnson: This election is about what we can do for you and your family. A majority Conservative government will end the uncertaint…', 'user': 'Emonkumar6', 'user_loc': 'mymensingh,bangladesh'}
{'hashtags': [], 'text': 'RT @ashindestad: @BaffledC The idea of voting Conservative for me is just very very mad LOL and I know people should vote for policies and…', 'user': 'ItsAbeo', 'user_loc': None}
{'hashtags': [], 'text': 'Now imagine a prominent conservative had been passing by and let loose:) swalwell wouldve fallen down crying assaul… https://t.co/UfutPwe8W9', 'user': 'Daddykins82', 'user_loc': 'United States'}
{'hashtags': [], 'text': '@patricklohlein @nickynoo007 @SteveBakerHW Maybe because Calvin Robinson is an ex-Conservative, and probably still… https://t.co/4OdQFrXVn7', 'user': 'ER_EJR', 'user_loc': 'West Hampstead'}
{'hashtags': [], 'text': 'RT @vote_dem: Fiscal conservative, family values, pro life. The @GOP is total bullshit and I thank twitt

{'hashtags': [], 'text': 'RT @BorisJohnson: This election is about what we can do for you and your family. A majority Conservative government will end the uncertaint…', 'user': 'ste_meehan', 'user_loc': None}
{'hashtags': [], 'text': 'RT @mel_faith1: Two conservative students attempting to promote their club on the campus of Binghamton University in Binghamton, New York…', 'user': 'cannonbalkid', 'user_loc': None}
{'hashtags': [], 'text': 'RT @Alykat24: Please use your vote wisely. You never know when you will need the NHS, if you vote conservative it won’t be there for long!…', 'user': 'Dealoftheday123', 'user_loc': None}
{'hashtags': [], 'text': 'RT @DFBHarvard: Half of @FoxNews is OK.  The other half is absolute CRAP!\n\nWith MSM 100% anti-Trump, anti-MAGA, anti-Conservative, anti-Chr…', 'user': 'doo2jen', 'user_loc': None}
{'hashtags': [], 'text': 'Trump Tested for Deliberate Poisoning of Food With “Time Delayed” Chemical Agent – White House Source… https://t.co/xsi2atXmDc', 'user'

{'hashtags': [], 'text': 'RT @ConservaMomUSA: WATCH-Radical leftists shut down scheduled talk by economist Dr. Art Laffer hosted by Binghamton University’s #Conserva…', 'user': 'Mark_Geertsma94', 'user_loc': 'Van down by the river'}
{'hashtags': [], 'text': '@SebastianEPayne @uncleporkie Good, it will be a farce. The leaders of the Conservative and Labour Party shouldn’t… https://t.co/5ffJCHESoI', 'user': 'Jamin2g', 'user_loc': None}
{'hashtags': [], 'text': 'RT @Independent: “The unweighted results show that only the Conservative Party received more positive than negative coverage across all new…', 'user': 'BuntsBilly', 'user_loc': None}
{'hashtags': ['Conservative', 'Ashfield', 'Labour', 'Ashfield'], 'text': 'RT @Anna_Soubry: The #Conservative candidate in #Ashfield is an odious sexist. I do not understand why the former #Labour MP for #Ashfield…', 'user': 'Bedri82547183', 'user_loc': None}
{'hashtags': [], 'text': "RT @clp_secretary: Council by-election last night St Mary's (Powys) 

{'hashtags': [], 'text': "Would have been good if our Conservatives had given even a moment's thought to holding an event like this.", 'user': 'AlexUsherHESA', 'user_loc': None}
{'hashtags': ['MySleeptweet'], 'text': 'RT @IWashington: #MySleeptweet You don’t wanna miss a Midnight Gutter Coon and a Conservative Thug on the same stage telling you the TRUTH…', 'user': 'nlackey777', 'user_loc': None}
{'hashtags': [], 'text': 'RT @KikkiPlanet: Oh hey, Canada. Kenney’s conservative government is firing the commissioner who is investigating their fraudulent leadersh…', 'user': 'PrairieCelt', 'user_loc': 'Winnipeg MB'}
{'hashtags': ['arronbanksleaks'], 'text': 'RT @Cromwell606: #arronbanksleaks \nWe now go live to the Conservative Party:\nhttps://t.co/V5xsWcUL8V', 'user': 'Lowery_1', 'user_loc': 'United Kingdom'}
{'hashtags': [], 'text': 'Leaked Xinjiang Papers Confirm The Chinese Communist Party Is Full Of Lying Murderers https://t.co/Q5ofFwVTQp', 'user': 'JadeNushuz4me', 'user_loc': 'New Jer

{'hashtags': [], 'text': "RT @Soutiam21: One of the best Conservative policies in this election is life means life. We've let perpetrators of the most hideous, murde…", 'user': 'AlanSha50343350', 'user_loc': None}
{'hashtags': [], 'text': "Now I'm sharing content from Tiana Lowe. This is proof that Fuentes is bad news for the conservative movement. I am… https://t.co/2DXuzlkVr6", 'user': 'arabbitorduck', 'user_loc': 'Texas, USA'}
{'hashtags': [], 'text': 'RT @Douglas4Moray: Myself and the @ScotTories have successfully campaigned for the last two years to mitigate the extra taxes the SNP have…', 'user': 'ThomasC95262363', 'user_loc': 'Scotland, United Kingdom'}
{'hashtags': [], 'text': 'RT @ananavarro: Groups are cutting ties with Michelle Malkin. Reportedly, she is being shunned for her support of racist, Holocaust-denying…', 'user': 'bloomsjo', 'user_loc': None}
{'hashtags': [], 'text': 'RT @carterforva: Dear 3% of Iowans who said Bernie is too conservative,\n\nI love you all. https:/

{'hashtags': [], 'text': 'RT @patel4witham: A Conservative majority government will give the police more powers to take weapons of the streets and take action agains…', 'user': 'GeoffreyClewes', 'user_loc': 'Bolton, England'}
{'hashtags': [], 'text': "@zennsfw1 Twitter can be so ridiculous! I don't see why they would do that. I'm sorry man.. I love your works so mu… https://t.co/bnYr1hxd5F", 'user': 'CelebAnalFakes', 'user_loc': None}
{'hashtags': [], 'text': 'RT @TrumpsPitBull: 😂😂How ironic that the @DNC isn’t pushing for their own agendas. It’s not just little ole conservative me that’s pissed b…', 'user': 'Jassy_Mom', 'user_loc': None}
{'hashtags': [], 'text': 'RT @mel_faith1: Two conservative students attempting to promote their club on the campus of Binghamton University in Binghamton, New York…', 'user': 'DennisWatkins15', 'user_loc': None}
{'hashtags': [], 'text': "In just three weeks, there's been more of a concerted effort of Conservative Inc's part to attack Fuentes then ther

KeyboardInterrupt: 

In [87]:
# Basic EDA

from collections import Counter
import ast

# Extract hashtags and put them in a list
list_hashtag_strings = [entry for entry in tweets.hashtags]
list_hashtag_lists = ast.literal_eval(','.join(list_hashtag_strings))
hashtag_list = [ht.lower() for list_ in list_hashtag_lists for ht in list_]

# Count most common hashtags
counter_hashtags = Counter(hashtag_list)
counter_hashtags.most_common(20)

AttributeError: 'DataFrame' object has no attribute 'hashtags'

In [69]:
from geopy.geocoders import Nominatim
import gmplot

geolocator = Nominatim()

# Go through all tweets and add locations to 'coordinates' dictionary
coordinates = {'latitude': [], 'longitude': []}
for count, user_loc in enumerate(tweets.location):
    try:
        location = geolocator.geocode(user_loc)
        
        # If coordinates are found for location
        if location:
            coordinates['latitude'].append(location.latitude)
            coordinates['longitude'].append(location.longitude)
            
    # If too many connection requests
    except:
        pass
    
# Instantiate and center a GoogleMapPlotter object to show our map
gmap = gmplot.GoogleMapPlotter(30, 0, 3)

# Insert points on the map passing a list of latitudes and longitudes
gmap.heatmap(coordinates['latitude'], coordinates['longitude'], radius=20)

# Save the map to html file
gmap.draw("python_heatmap.html")

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  after removing the cwd from sys.path.


AttributeError: 'DataFrame' object has no attribute 'location'